In [ ]:
"""This ipynb file convert using the whole pic semantic mask to create smaller pathces"""

In [1]:
import utils
from prostate_dataset import ProstateDataset
import numpy as np
import os
import scipy.io

In [2]:
def _ann_padding(annotation_org):
        ann_pad = np.lib.pad(annotation_org, ((150, 150), (150, 150), (0, 0)), 'symmetric') 
        return ann_pad

In [3]:
def _crop_split_image(annotation_org, class_ids, dataset_dir\
                                     height = 600, width = 600, \
                                     image_id = 0, save = False):
        imgheight, imgwidth, _ = annotation_org.shape
        value = 1
        current_image = 0
        for i in range(0, imgheight - 301, 300):
            for j in range(0, imgwidth - 301, 300):
                new_class_ids = []
                flag = False
                for index in range(len(class_ids)):
                    ann_temp = annotation_org[i : i + height, j : j + width, index]
                    if value in ann_temp:
                        new_class_ids.append(class_ids[index])
                        new_instance_mask = np.concatenate((new_instance_mask, np.expand_dims(ann_temp, axis = -1)),\
                                                           axis = -1)\
                        if flag else np.expand_dims(ann_temp, axis = -1)
                        flag = True
                if save:
                    res_dict = {'segmentation': new_instance_mask, 'class_ids': new_class_ids}
                    ann_filename = str(image_id).zfill(4) + '_' + str(current_image).zfill(4) + '_instance.mat'
                    filepath = os.path.join(dataset_dir, 'masks_instance_mod_16/')
                    scipy.io.savemat(filepath + \
                                     ann_filename, res_dict)
                    current_image += 1

In [ ]:
data_path = '/data/wenyuan/Path_R_CNN/Data_Pre_Processing'
dataset_dir = os.path.join(data_path, 'cedars-224')
dataset = ProstateDataset(dataset_dir)

start_id = 470
end_id = 513
display_step = 20

for image_id in range(start_id, end_id):
    mask, class_ids = dataset.read_instance_ann(image_id)
    if mask.ndim == 2:
        mask = np.expand_dims(mask, axis = -1)
        class_ids = [class_ids]
    mask = _ann_padding(mask)
    _crop_split_image(mask, class_ids, dataset_dir, image_id = image_id, save = True)
    if image_id % display_step ==0:
        print('Done converting image %d!' %image_id)

Done converting image 480!
